In [13]:
from config.Neo_connect import Neo4jConnection
import pandas as pd
import config.db_settings as db_config

# stablish connection with neo4j
conn = Neo4jConnection(uri="bolt://localhost:7687", user=db_config.USER, pwd=db_config.PASSWORD)
#result = conn.query('create database {}'.format(DATABASE))

In [14]:
#create nodes

#-----------------------------------
node_queries = [
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data/stock_data.csv' as row FIELDTERMINATOR ',' CREATE (:stock {symbol:row.symbol,lastPrice :row.lastPrice,
dayHigh: row.dayHigh,dayLow: row.dayLow,closePrice:row.closePrice,open:row.open,timestamp_ID:row.timestamp_ID, stock_market_ID:row.stock_market_ID,
company_ID:row.company_ID }) return row.symbol;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data/stock_data.csv' as row FIELDTERMINATOR ',' CREATE (:company {company_ID:row.company_ID,symbol:row.symbol,companyName:row.companyName }) return row.company_ID;
''',

'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data/stock_market.csv' as row FIELDTERMINATOR ',' CREATE (:stock_market {ID:row.ID, country:row.country, stock_name:row.stock_name }) return row.country;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data/time.csv' as row FIELDTERMINATOR ',' CREATE (:time {ID:row.ID, year:row.year, month:row.month, day:row.day, hour:row.hour, timestamp:row.timestamp}) return row.ID;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data/tweets_users.csv' as row FIELDTERMINATOR ',' CREATE (:tweets {tweet_ID:row.tweet_ID,symbol:row.company_name,text:row.text,support_Count:row.support_Count,rank:row.rank,sentiment:row.sentiment,polarity:row.polarity,company_ID:row.company_ID,timestamp_ID:row.timestamp_ID}) return row.tweet_ID;
''',
'''
LOAD CSV WITH HEADERS FROM 'file:/Users/himanshu/SDM_Project/data/tweets_users.csv' as row FIELDTERMINATOR ',' CREATE (:tweet_users {tweet_ID:row.tweet_ID, user: row.user,mentioned_user:row.mentioned_user}) return row.user;
''']


for i, nd_query in enumerate(node_queries):
    result = conn.query(nd_query, db=db_config.DATABASE)
    # print("query {} executed \n {} \n".format(i+1,nd_query))


Query failed: {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: At /Users/himanshu/SDM_Project/data/tweets_users.csv @ position 8745 -  there's a field starting with a quote and whereas it ends that quote there seems to be characters in that field after that ending quote. That isn't supported. This is what I read: '"#NCTDREAM Breaks Personal Record With Over 2 Million Stock Pre-Orders For ""G"G'}
Query failed: {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: At /Users/himanshu/SDM_Project/data/tweets_users.csv @ position 8745 -  there's a field starting with a quote and whereas it ends that quote there seems to be characters in that field after that ending quote. That isn't supported. This is what I read: '"#NCTDREAM Breaks Personal Record With Over 2 Million Stock Pre-Orders For ""G"G'}


In [15]:
#make_relations
#-----------------------------------------------

relation_queries = [
'''
MATCH (st:stock {c_id: company_ID}),(company:company {company_id: company_ID})
CREATE (st)-[r:belongs_to]->(company) return r
''',
'''
MATCH (tweet:tweets {c_id: company_ID}),(company:company {company_id: company_ID})
CREATE (tweet)-[r:is_related_to]->(company) return r
''',
'''
MATCH (st:stock {t_id: timestamp_ID}),(time:time {ID: ID})
CREATE (st)-[r:at]->(time) return r
''',
'''
MATCH (tweet:tweets {t_id: timestamp_ID}),(time:time {ID: ID})
CREATE (tweet)-[r:at_this]->(time) return r
''',
'''
MATCH (st:stock {sm_id:stock_market_ID }),(st_market:stock_market {ID: ID})
CREATE (st)-[r:is_in]->(st_market) return r
''',    
'''
MATCH (tweet:tweets {t_id: tweet_ID}),(users:tweet_users {ID: tweet_ID})
CREATE (tweet)-[r:contains]->(users) return r
''']


for j,re_query in enumerate(relation_queries):
    result = conn.query(re_query, db=db_config.DATABASE)
    print("query {} executed \n {} \n".format(j+1,re_query))











#=======================================================================================
# # creating index on publisher node
# ind_q = '''CREATE INDEX conf_journ_idx FOR (pub:publisher) ON (pub.type)'''
# result = conn.query(ind_q, db=db_config.DATABASE)
# print("index created on publisher node on attribute type")



# q = ''' MATCH (n:article) RETURN n LIMIT 25'''
# result = conn.query(q, db=DATABASE)
# # print(result[1].data())
# dtf_data = pd.DataFrame([_.data()['n'] for _ in result])
# print(dtf_data)
# loading data from CSV
#---------------------------------------------------------------------


Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `company_ID` not defined (line 2, column 24 (offset: 24))
"MATCH (st:stock {c_id: company_ID}),(company:company {company_id: company_ID})"
                        ^}
query 1 executed 
 
MATCH (st:stock {c_id: company_ID}),(company:company {company_id: company_ID})
CREATE (st)-[r:belongs_to]->(company) return r
 

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `company_ID` not defined (line 2, column 28 (offset: 28))
"MATCH (tweet:tweets {c_id: company_ID}),(company:company {company_id: company_ID})"
                            ^}
query 2 executed 
 
MATCH (tweet:tweets {c_id: company_ID}),(company:company {company_id: company_ID})
CREATE (tweet)-[r:is_related_to]->(company) return r
 

Query failed: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `timestamp_ID` not defined (line 2, column 24 (offset: 24))
"MATCH (st:stock {t_id: timestamp_ID}),(time:time {ID: 